 # Import necessary packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Raw DataFrames

In [2]:
# global land temperature data by city, with latitude/longitude values
filename = 'Global-Land-Temperatures-By-City.csv'
temp_df = pd.read_csv(filename)

In [3]:
# US federal emergency data, join on county
filename1 = 'federal_emergencies.csv'
disaster_df = pd.read_csv(filename1)

In [4]:
# data of latitude/longitude and county to merge two dataframes
filename2 = 'zip_codes_states.csv'
us_join = pd.read_csv(filename2)

# Cleaning Steps

In [5]:
# temp_df: drop all countries except US and drop NaN values
temp_us = temp_df[temp_df['Country'] == 'United States'].dropna()

In [6]:
# temp_df: create new lat/long columns dropping NESW direction
temp_us['lat_n'] = [float(lat[:-1]) if lat[-1]=='N' else -1*float(lat[:-1]) for lat in temp_us.loc[:,'Latitude']]
temp_us['lon_n'] = [float(lon[:-1]) if lon[-1]=='E' else -1*float(lon[:-1]) for lon in temp_us.loc[:,'Longitude']]

In [7]:
# unique coordinates in the temp_us dataframe
temp_us_coords = temp_us[['lat_n','lon_n']].drop_duplicates()

In [8]:
# function adds a column with pythagorean theorem 
def coord2loc(coords):
    us2 = us_join.copy()
    us2['dist'] = ((us2.latitude-coords.lat_n)**2+(us2.longitude-coords.lon_n)**2)**(1/2)
    state = us2.loc[us2.dist==min(us2.dist)]['state'].values[0]
    county = us2.loc[us2.dist==min(us2.dist)]['county'].values[0]
    return([coords.lat_n,coords.lon_n,state,county])

In [ ]:
# create dataframe to join on between
join = pd.DataFrame([coord2loc(coords[1]) for coords in temp_us_coords.iterrows()])
join.columns = ['lat_n','lon_n','state','county']

In [ ]:
# merge temp_df and us_join
temp_county = pd.merge(temp_us, join, how='left', on = ['lat_n', 'lon_n'])

In [ ]:
# add "County" to the end of the county names to join on the disaster dataframe
temp_county['countyname'] = temp_county.county +' County'

In [ ]:
temp_county['dt'] = pd.to_datetime(temp_county['dt'], format='%Y/%m/%d')
temp_county['year'] = temp_county['dt'].dt.year

In [ ]:
temp_county.head()

In [ ]:
# Extract month number for each row
temp_county_seasons = temp_county.copy()
temp_county_seasons['month'] = temp_county_seasons['dt'].dt.month

In [ ]:
# Assign seasons to each date
temp_county_seasons['month'] = temp_county_seasons['month'].astype(str).astype(int)
season_map = {12: 'Winter', 1: 'Winter', 2: 'Winter', 
              3: 'Spring', 4: 'Spring', 5: 'Spring', 
              6: 'Summer', 7: 'Summer', 8: 'Summer', 
              9: 'Fall', 10: 'Fall', 11: 'Fall'}
def mapper(month):
    return season_map[month]
temp_county_seasons['season'] = temp_county_seasons['month'].apply(mapper)

In [ ]:
# Assign United States common regions to the states
## Midwest Region
east_north_central_midwest_region = ['IL','IN','MI','OH','WI']
d1 = dict.fromkeys(east_north_central_midwest_region, 'east north central midwest region')

west_north_central_midwest_region = ['IA','KS','MO','MN','ND','SD','NE']
d2 = dict.fromkeys(west_north_central_midwest_region, 'west north central midwest region')

## Northeast Region
new_england_northeast_region = ['CT','ME','MA','NH','RI','VT']
d3 = dict.fromkeys(new_england_northeast_region, 'new england northeast region')

midatlantic_northeast_region = ['NY','PA','NJ']
d4 = dict.fromkeys(midatlantic_northeast_region, 'midatlantic northeast region')

## West Region
pacific_west_region = ['AK','OR','WA','CA','HI']
d5 = dict.fromkeys(pacific_west_region, 'pacific west region')

mountain_west_region = ['AZ','CO','NM','UT','NV','WY','ID','MT']
d6 = dict.fromkeys(mountain_west_region, 'mountain west region')

## South Region
west_south_central_south_region = ['AR','LA','OK','TX']
d7 = dict.fromkeys(west_south_central_south_region, 'west south central south region')

east_south_central_south_region = ['AL','MS','TN','KY']
d8 = dict.fromkeys(east_south_central_south_region, 'east south central south region')

south_atlantic_south_region = ['WV','MD','DC','DE','VA','NC','SC','GA','FL']
d9 = dict.fromkeys(south_atlantic_south_region, 'south atlantic south region')


In [ ]:
temp_county_region = temp_county_seasons.copy()
d = {**d1, **d2, **d3, **d4, **d5, **d6, **d7, **d8, **d9}
temp_county_region['region'] = temp_county_region['state'].map(d)

In [ ]:
temp_county_region['date_delta'] = (temp_county_region['dt'] - temp_county_region['dt'].min())  / np.timedelta64(1,'D')


#### Join for RF Model

In [ ]:
temp_county_region.head()

In [ ]:
# disaster data - create year column
disasterdf = disaster_df.copy()
disasterdf['Declaration Date'] = pd.to_datetime(disasterdf['Declaration Date'], format='%m/%d/%Y')
disasterdf['Year'] = disasterdf['Declaration Date'].dt.year

# create region column
disasterdf['Region'] = disasterdf['State'].map(d)

# select columns for merge 
disasterdf = disasterdf[['Declaration Type','Declaration Date','State','County','Disaster Type','Year','Region']]


In [ ]:
# disaster data - delete duplicate listing of disasters on the same day (ie. unique disaster per day per state)
disasterdf = disasterdf.drop_duplicates(subset=['Declaration Date','Disaster Type','State','Year']).sort_values('Declaration Date')

# select top 5 disasters
disasterdf = disasterdf[disasterdf['Disaster Type'].isin(['Tornado','Flood','Fire','Hurricane','Storm'])]

# create new disaster count column 
disasterdf['Disaster Count'] = disasterdf['Disaster Type']


In [ ]:
# disaster data groupby, get disaster count by region, disaster type, and year
disasterdf = disasterdf.groupby(['Region','Disaster Type','Year'])[['Disaster Count']].count().unstack(fill_value=0).stack().reset_index()
disasterdf

In [ ]:
# copy of temperature data to organize for the join
tempdf = temp_county_region.copy()
tempdf = tempdf.sort_values('dt')

In [ ]:
# select columns for join
tempdf = tempdf[['year','AverageTemperature','season','region']]

In [ ]:
# temperature data - feature engineer temperature by region and year
tempdf = tempdf.groupby(['region','year']).agg({'AverageTemperature': ['mean','min','max','std']}).unstack(fill_value=0).stack().reset_index()
tempdf.columns = ["_".join(x) for x in tempdf.columns.ravel()]

In [ ]:
# rename columns in temperature data frame
tempdf = tempdf.rename(columns={"region_": "region", "year_": "year"});

In [ ]:
tempdf.head()

In [ ]:
# create dataframe for seasonal temperature by year and region
season = temp_county_region.pivot_table('AverageTemperature', index=['year','region'], columns='season', fill_value='NaN').reset_index()
season.head()

In [ ]:
# join the temperature dataframe with the seasonal temperatures to obtain more features
join_dataframe = pd.merge(tempdf, season, left_on=['region','year'], right_on=['region','year'], how='left')
join_dataframe

In [ ]:
disasterdf.head()

In [ ]:
# join temperature and disaster dataframe 
joindf = pd.merge(disasterdf, join_dataframe, left_on=['Region','Year'], right_on=['region','year'], how='left')

In [ ]:
# drop duplicate columns
joindf = joindf.drop(columns = ['region','year'])

In [ ]:
joindf.head()

In [ ]:
joindf.info()

# Random Forest for disaster/region pairs

In [ ]:
disasters = joindf['Disaster Type'].unique()
regions = joindf['Region'].unique()

combined = [(s, f) for s in regions for f in disasters]

In [ ]:
joindf = joindf.set_index(['Region','Disaster Type']).dropna()

In [ ]:
joindf.loc[('east north central midwest region','Fire')]

In [ ]:
for k in combined:
    try:
        print(joindf.loc[k])
    except:
        print(k)

In [ ]:
def randforest(key):
    
    # random forest regression
    X = joindf.loc[key].drop(['Disaster Count'], axis=1).values
    y = joindf.loc[key]['Disaster Count'].values
    names = join_dataframe[['AverageTemperature_mean','AverageTemperature_min','AverageTemperature_max','AverageTemperature_std','Fall','Spring','Summer','Winter']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

    # Create the regressor: 
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

    # Fit the regressor to the training data
    rf.fit(X_train, y_train)

    # Predict on the test data: y_pred
    y_pred = rf.predict(X_test)

    print('R^2 or Score:', rf.score(X_test, y_test))
    print ("Features sorted by their importance:", sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), reverse=True))


In [ ]:
for k in combined:
    try:
        print(k, randforest(k))
    except:
        print(k, 'does not exist')

The four region/disaster pairing with the highest score are:
   
   east nort central midwest region, storm
    
   mountain west region, fire
   
   mountain west region, storm
   
   pacific west region, fire

### temperature prediction

I will use those 4 region/disaster pairings to focus on for predicting average temperature and disaster counts.

In [ ]:
columns = joindf.columns.drop('Disaster Count')

# linear regression to obtain independent variable predictions to use for random forest
def linearreg(key, column, year):
    X = joindf.loc[key][['Year']].values
    y = joindf.loc[key][column].values

    X = X.reshape(-1,1)
    y = y.reshape(-1,1)

    model = linear_model.LinearRegression()
    model.fit(X, y)
    
    print(model.predict(year))
    #plt.scatter(X, y,color='r')

    #plt.plot(X, model.predict(X),color='k')
    #plt.show()

In [ ]:
def randforestpredictor(x_var, key):
    
    # random forest regression
    X = joindf.loc[key].drop(['Disaster Count'], axis=1).values
    y = joindf.loc[key]['Disaster Count'].values
    names = join_dataframe[['AverageTemperature_mean','AverageTemperature_min','AverageTemperature_max','AverageTemperature_std','Fall','Spring','Summer','Winter']]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

    # Create the regressor: 
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

    # Fit the regressor to the training data
    rf.fit(X_train, y_train)

    # Predict on the test data: y_pred
    y_var = rf.predict(x_var)
    
    print(y_var)

#### east north central midwest region / storm 

In [ ]:
# predict year 2070
for col in columns:
    print(linearreg(('east north central midwest region', 'Storm'), col, 2070))

In [ ]:
# rf model prediction using these values 
# year 2070
x_2070 = [[2070, 11.8441235, -6.04739043, 26.66335563, 9.35311371, 13.31567534, 11.38216591, 23.25243497, -0.56003033]]

In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2070, ('east north central midwest region', 'Storm'))

In [ ]:
# predict year 2120
for col in columns:
    print(linearreg(('east north central midwest region', 'Storm'), col, 2120))

In [ ]:
# rf model prediction using these values 
x_2120 = [[2120, 12.80583185, -4.78466965, 27.34652591, 9.15965208, 14.18573028, 12.53277554, 24.01557468, 0.51006306]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2120, ('east north central midwest region', 'Storm'))

In [ ]:
# predict year 2220
for col in columns:
    print(linearreg(('east north central midwest region', 'Storm'), col, 2220))

In [ ]:
# rf model prediction using these values 
x_2220 = [[2220, 14.72924853, -2.25922808, 28.71286647, 8.77272884, 15.92584018, 14.8339948, 25.54185409, 2.65024985]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2220, ('east north central midwest region', 'Storm'))

#### mountain west region / fire

In [ ]:
# predict year 2070
for col in columns:
    print(linearreg(('mountain west region', 'Fire'), col, 2070))

In [ ]:
# rf model prediction using these values 
# year 2070
x_2070 = [[2070, 16.1550363, -6.30327229, 33.96463585, 10.28919724, 17.18435082, 15.6861039, 26.97222201, 4.73691442]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2070, ('mountain west region', 'Fire'))

In [ ]:
# predict year 2120
for col in columns:
    print(linearreg(('mountain west region', 'Fire'), col, 2120))

In [ ]:
# rf model prediction using these values 
x_2120 = [[2120, 17.03366165, -5.15347589, 34.68687805, 10.4455514, 18.29039419, 16.95982956, 27.84635199, 4.97668426]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2120, ('mountain west region', 'Fire'))

In [ ]:
# predict year 2220
for col in columns:
    print(linearreg(('mountain west region', 'Fire'), col, 2220))

In [ ]:
# rf model prediction using these values 
x_2220 = [[2220, 18.79091235, -2.85388308, 36.13136245, 10.75825971, 20.50248093, 19.50728087, 29.59461194, 5.45622396]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2220, ('mountain west region', 'Fire'))

#### mountain west region / storm

In [ ]:
# predict year 2070
for col in columns:
    print(linearreg(('mountain west region', 'Storm'), col, 2070))

In [ ]:
# rf model prediction using these values 
# year 2070
x_2070 = [[2070, 16.1550363, -6.30327229, 33.96463585, 10.28919724, 17.18435082, 15.6861039, 26.97222201, 4.73691442]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2070, ('mountain west region', 'Storm'))

In [ ]:
# predict year 2120
for col in columns:
    print(linearreg(('mountain west region', 'Storm'), col, 2120))

In [ ]:
# rf model prediction using these values 
x_2120 = [[2120, 17.03366165, -5.15347589, 34.68687805, 10.4455514, 18.29039419, 16.95982956, 27.84635199, 4.97668426]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2120, ('mountain west region', 'Storm'))

In [ ]:
# predict year 2220
for col in columns:
    print(linearreg(('mountain west region', 'Storm'), col, 2220))

In [ ]:
# rf model prediction using these values 
x_2220 = [[2220, 18.79091235, -2.85388308, 36.13136245, 10.75825971, 20.50248093, 19.50728087, 29.59461194, 5.45622396]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2220, ('mountain west region', 'Storm'))

#### pacific west region / fire

In [ ]:
# predict year 2070
for col in columns:
    print(linearreg(('pacific west region', 'Fire'), col, 2070))

In [ ]:
# rf model prediction using these values 
# year 2070
x_2070 = [[2070, 16.22593332, -11.75978244, 30.15020206, 6.23724319, 17.80460639, 15.37083785, 22.22981778, 9.61202119]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2070, ('pacific west region', 'Fire'))

In [ ]:
# predict year 2120
for col in columns:
    print(linearreg(('pacific west region', 'Fire'), col, 2120))

In [ ]:
# rf model prediction using these values 
x_2120 = [[2120, 17.05502805, -8.660967, 31.02268488, 6.322795, 18.80943167, 16.51741249, 23.06318462, 10.00196381]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2120, ('pacific west region', 'Fire'))

In [ ]:
# predict year 2220
for col in columns:
    print(linearreg(('pacific west region', 'Fire'), col, 2220))

In [ ]:
# rf model prediction using these values 
x_2220 = [[2220, 18.71321751, -2.46333612, 32.7676505, 6.49389862, 20.81908223, 18.81056178, 24.7299183, 10.78184903]]


In [ ]:
# random forest using temperature values to predict disaster counts
randforestpredictor(x_2220, ('pacific west region', 'Fire'))